# Azure ML Compute Python SDK

description: overview of the AML Compute Python SDK

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace=ws)
# AmlCompute.supported_vmsizes(workspace = ws, location='southcentralus')

### Provision as a persistent compute target (Basic)

You can provision a persistent AmlCompute resource by simply defining two parameters thanks to smart defaults. By default it autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

You can also specify additional properties or change defaults while provisioning AmlCompute using a more advanced configuration. This is useful when you want a dedicated cluster of 4 nodes (for example you can set the min_nodes and max_nodes to 4), or want the compute to be within an existing VNet in your subscription.

In addition to `vm_size` and `max_nodes`, you can specify:
* `min_nodes`: Minimum nodes (default 0 nodes) to downscale to while running a job on AmlCompute
* `vm_priority`: Choose between 'dedicated' (default) and 'lowpriority' VMs when provisioning AmlCompute. Low Priority VMs use Azure's excess capacity and are thus cheaper but risk your run being pre-empted
* `idle_seconds_before_scaledown`: Idle time (default 120 seconds) to wait after run completion before auto-scaling to min_nodes
* `vnet_resourcegroup_name`: Resource group of the **existing** VNet within which AmlCompute should be provisioned
* `vnet_name`: Name of VNet
* `subnet_name`: Name of SubNet within the VNet
* `admin_username`: Name of Admin user account which will be created on all the nodes of the cluster
* `admin_user_password`: Password that you want to set for the user account above
* `admin_user_ssh_key`: SSH Key for the user account above. You can specify either a password or an SSH key or both
* `remote_login_port_public_access`: Flag to enable or disable the public SSH port. If you dont specify, AmlCompute will smartly close the port when deploying inside a VNet
* `identity_type`: Compute Identity type that you want to set on the cluster, which can either be SystemAssigned or UserAssigned
* `identity_id`: Resource ID of identity in case it is a UserAssigned identity, optional otherwise


In [ ]:
from random import randint
from azureml.core.compute import AmlCompute, ComputeTarget

# name must be unique within a workspace
ct_name = f"ct-{str(randint(10000, 99999))}-concept"

if ct_name in ws.compute_targets:
    ct = ws.compute_targets[ct_name]
    ct.delete()
    ct.wait_for_completion(show_output=True, is_delete_operation=True)

compute_config = AmlCompute.provisioning_configuration(
    vm_size="STANDARD_D2_V2", max_nodes=4
)

ct = ComputeTarget.create(ws, ct_name, compute_config)
ct.wait_for_completion(show_output=True)

In [ ]:
# get_status() gets the latest status of the AmlCompute target
ct.get_status().serialize()

In [ ]:
# list_nodes() gets the list of nodes on the cluster with status, IP and associated run
ct.list_nodes()

In [ ]:
# update() takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
# ct.update(min_nodes=1)
# ct.update(max_nodes=10)
# ct.update(idle_seconds_before_scaledown=300)
# ct.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [ ]:
# delete() is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name
ct.delete()
ct.wait_for_completion(show_output=True, is_delete_operation=True)